# Preprocessing pipeline for reliability bootstrapping
1. Extract & cut spike trains
2. Compute filtered spike signals & mean-centering
3. Compute firing rates

In [1]:
import sys
sys.path.append('../../library')

### 1. Extract & cut spike trains

Extracts (excitatory) spike trains in format compatible with "toposample" pipeline

In [2]:
from extract import run_extraction

In [3]:
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_Baseline/'
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnRewired_mc2EE_Order1'
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnRewired_mc2EE_Order2'
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnRewired_mc2EE_Order3'
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnRewired_mc2EE_Order4'
campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnRewired_mc2EE_Order5'

# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_Baseline_2023-07__WRONG_SIMULATOR_VERSION__/'

# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA'
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_2'
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_3'

# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipRemoval_Unstruct-3/'
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipRemoval_Unstruct-2/'
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipRemoval_Unstruct-1/'
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipRemoval_Unstruct-0/'
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipRemoval_StructDim456'
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipRemoval_StructDim56_456'
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipRemoval_StructDim56'

# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_OutConnsRemoved_BlockDesign_Struct'
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_OutConnsRemoved_BlockDesign_Rnd'
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipConnsRemoved_BlockDesign_Struct'
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipConnsRemoved_BlockDesign_Rnd'

sim_paths, working_dir = run_extraction(campaign_path, working_dir_name='working_dir')
num_sims = len(sim_paths)

/gpfs/bbp.cscs.ch/home/pokorny/BbpWorkflowKernel/lib/python3.9/site-packages/tables/file.py:411: UserWarning: a closed node found in the registry: ``/neuron_info/meta/values_block_3/meta/_i_table``
  warnings.warn("a closed node found in the registry: "
/gpfs/bbp.cscs.ch/ssd/apps/bsd/2022-01-10/stage_applications/install_gcc-11.2.0-skylake/py-numpy-1.19.5-gjm7kk/lib/python3.9/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


INFO: 30 spike files written to "/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/bbp_workflow/c37dc89e-57bf-43be-9f30-df345b0cac39/working_dir"


### 2. Compute filtered spike signals & mean-centering

Runs preprocessing (filtering, w/o mean-centering) of (excitatory) spike trains [PARALLEL IMPLEMENTATION]

In [4]:
from preprocess import run_preprocessing, merge_into_h5_data_store

In [5]:
# Run preprocessing
spike_file_names = [f'raw_spikes_exc_{idx}.npy' for idx in range(num_sims)]

run_preprocessing(working_dir, spike_file_names, sigma=10.0, mean_centered=False, pool_size=10)

Finished preprocessing in 119.974s


In [6]:
# Merge individual preprocessed spike files into .h5 data store
#   split_by_gid==False ... Datasets per sim
#   split_by_gid==True  ... Datasets per sim & GID
tmp_file_names = [f'spike_signals_exc_{idx}__tmp__.npz' for idx in range(num_sims)]

h5_file = merge_into_h5_data_store(working_dir, tmp_file_names, data_store_name='processed_data_store', split_by_gid=False)

100%|██████████| 30/30 [16:10<00:00, 32.34s/it]

INFO: 30 files merged into "/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/bbp_workflow/c37dc89e-57bf-43be-9f30-df345b0cac39/working_dir/processed_data_store.h5"


### 3. Compute firing rates

Runs firing rate extraction based on mean inverse inter-spike interval of (excitatory) spike trains [PARALLEL IMPLEMENTATION]

In [7]:
from preprocess import run_rate_extraction, merge_rates_to_h5_data_store

In [8]:
# Run rate extraction
spike_file_names = [f'raw_spikes_exc_{idx}.npy' for idx in range(num_sims)]

run_rate_extraction(working_dir, spike_file_names, pool_size=30)

/gpfs/bbp.cscs.ch/ssd/apps/bsd/2022-01-10/stage_applications/install_gcc-11.2.0-skylake/py-numpy-1.19.5-gjm7kk/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/gpfs/bbp.cscs.ch/ssd/apps/bsd/2022-01-10/stage_applications/install_gcc-11.2.0-skylake/py-numpy-1.19.5-gjm7kk/lib/python3.9/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/gpfs/bbp.cscs.ch/ssd/apps/bsd/2022-01-10/stage_applications/install_gcc-11.2.0-skylake/py-numpy-1.19.5-gjm7kk/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/gpfs/bbp.cscs.ch/ssd/apps/bsd/2022-01-10/stage_applications/install_gcc-11.2.0-skylake/py-numpy-1.19.5-gjm7kk/lib/python3.9/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double

Finished firing rate extraction in 46.159s


In [9]:
# Merge individual rate files into .h5 data store
tmp_file_names = [f'firing_rates_exc_{idx}__tmp__.npz' for idx in range(num_sims)]

h5_file = merge_rates_to_h5_data_store(working_dir, tmp_file_names, data_store_name='processed_data_store', do_overwrite=False)

100%|██████████| 30/30 [00:00<00:00, 652.54it/s]

INFO: 30 files merged and added to "/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/bbp_workflow/c37dc89e-57bf-43be-9f30-df345b0cac39/working_dir/processed_data_store.h5"


### 4. Extract deleted connections (optional)

Adds information about deleted connections to data store (e.g., for block-based designs)

ℹ️ Requires .npy files containing deleted connections within sim folders

In [2]:
import os
from preprocess import merge_removed_conns_to_h5_data_store

In [ ]:
campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_OutConnsRemoved_BlockDesign_Struct'
merge_removed_conns_to_h5_data_store(os.path.join(campaign_path, 'working_dir', 'processed_data_store.h5'), campaign_path, 'outgoing_conns_removed.npy')

campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_OutConnsRemoved_BlockDesign_Rnd'
merge_removed_conns_to_h5_data_store(os.path.join(campaign_path, 'working_dir', 'processed_data_store.h5'), campaign_path, 'outgoing_conns_removed.npy')

campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipConnsRemoved_BlockDesign_Struct'
merge_removed_conns_to_h5_data_store(os.path.join(campaign_path, 'working_dir', 'processed_data_store.h5'), campaign_path, 'conns_removed.npy')

campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipConnsRemoved_BlockDesign_Rnd'
merge_removed_conns_to_h5_data_store(os.path.join(campaign_path, 'working_dir', 'processed_data_store.h5'), campaign_path, 'conns_removed.npy')


__HOW TO LOAD PROCESSED SPIKE SIGNALS, META-INFO, AND RATES FROM .H5 DATA STORE:__
~~~
h5_store = h5py.File(h5_file, 'r')
print(f'Groups/Datasets: {list(h5_store.keys())}')

t_bins = np.array(h5_store['t_bins'])
gids = np.array(h5_store['gids'])
firing_rates = np.array(h5_store['firing_rates'])
mean_centered = np.array(h5_store['mean_centered']).tolist()
sigma = np.array(h5_store['sigma']).tolist()

print(f'Spike signals per sims: {list(h5_store["spike_signals_exc"].keys())}')
if split_by_gid == True:
    print(f'Spike signals within sim <SIM_IDX>: {list(h5_store["spike_signals_exc/sim_<SIM_IDX>"].keys())}')
    spike_signal = np.array(h5_store[f'spike_signals_exc/sim_{<SIM_IDX>}/gid_{<GID>}'])
else:
    spike_signals = np.array(h5_store[f'spike_signals_exc/sim_{<SIM_IDX>}'])

if 'outgoing_conns_removed' in h5_store: # Sources of removed outgoing connections
    outgoing_conns_removed = np.array(h5_store[f'outgoing_conns_removed/sim_{<SIM_IDX>}'])

if 'conns_removed' in h5_store: # Sources/targets of removed connections
    conns_removed = np.array(h5_store[f'conns_removed/sim_{<SIM_IDX>}'])

h5_store.close()
~~~